In [19]:
from window_data import save_window_data

import torch
from nnsight import LanguageModel
from sae_lens import SAE
from datasets import load_dataset
from transformer_lens import utils


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dtype = torch.bfloat16

In [10]:
model = LanguageModel("google/gemma-2-2b", device_map=device, torch_dtype=dtype)

c:\Users\jacob\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\jacob\.cache\huggingface\hub\models--google--gemma-2-2b. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [11]:
layer = 13
hook_pt = f"blocks.{layer}.hook_resid_post"
submodule = model.model.layers[layer] # nnsight will grab activations at the OUTPUT of this submodule
n_ctx = 1024

In [15]:
# load gemma scope SAE
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = "gemma-scope-2b-pt-res-canonical",
    sae_id = f"layer_{layer}/width_16k/canonical",
)
sae = sae.to(device).to(dtype)

In [20]:
# get OWT data
eval_data = load_dataset("stas/openwebtext-10k", split="train")
tokenized_data = utils.tokenize_and_concatenate(eval_data, model.tokenizer, max_length=n_ctx)
tokenized_data = tokenized_data.shuffle(42)
eval_tokens = tokenized_data["tokens"]
eval_tokens.shape

Map (num_proc=10): 100%|██████████| 10000/10000 [00:26<00:00, 381.87 examples/s]


torch.Size([10683, 1024])

In [23]:
save_window_data(
    model,
    submodule,
    sae,
    eval_tokens[:, :64],
    save_dir = "/Users/jacob/vscode-stuff/features-over-time",
    num_contexts = 20, #eval_tokens.shape[0],
    batch_size = 8*2,
    window_length = 64,
    stride = 16,
    num_tokens_to_generate = 0,
    temperature = 1.0,
    return_feature_acts = False,
    compress = True
)

Processing batches:   0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: unknown device type for autocast in get_autocast_dispatch_key_from_device_type